In [ ]:
import os
import sys
sys.path.append("/mnt/scratch/henrypurcell/N3AS/Emu_scripts/spinflip")
import yt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import h5py
from constants import c, M_p, M_3flavor, G, hbar
import spin_flip_tools as sft
import gellmann as gm
import four_current as fc
import matrix
import constants
import diagonalizer as dg
import time_plots as tp
import merger_grid as mg
import glob
import concurrent
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator,LogLocator)
from matplotlib import cm, colors
import matplotlib.axes as ax
from mpl_toolkits.mplot3d import Axes3D
from four_current import four_current, read_gradients
from matrix import visualizer
import argparse
#import latex
from itertools import product

# For ease of development, reload the relevant modules to make sure they are up to date.
import importlib
importlib.reload(constants)
importlib.reload(sft)
importlib.reload(gm)
importlib.reload(fc)
importlib.reload(matrix)
importlib.reload(dg)
importlib.reload(tp)
importlib.reload(mg)

################
# plot options #
################
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=True)

mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.size'] = 4
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.top'] = True

mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['axes.grid'] = False

np.set_printoptions(precision=16)

#####################
###plot parameters###
#####################



#location to save png's
plot_output_file = "/mnt/scratch/henrypurcell/N3AS/5-figures_final/"

#resolutions
resolution = 10

merger_data_filename = "/mnt/scratch/shared/2-orthonormal_distributions/model_rl0_orthonormal_rotated.h5"
emu_data_loc = "/mnt/scratch/shared/3-Henry_NSM_box_bestpoints/"
gradient_filename = "/mnt/scratch/shared/4-gradients/gradients_start_i117-134_j64-78_k14-17.h5"
gradient_filename_end = "/mnt/scratch/shared/4-gradients/gradients_end_i117-134_j64-78_k14-17.h5"
#3 z-values of cross sections to plot
zvals = [8,15,99]

#location of specific point to analyze
location=[122,70,15] 
locations = [[105,130,98], #for extra bands plots
             [125,128,98],
             [78,72,98],
             [75,69,98],
             ]
emu_filename = emu_data_loc + "i{:03d}".format(location[0])+"_j{:03d}".format(location[1])+"_k{:03d}".format(location[2])+"/allData.h5"

#boundaries of regions over which gradients have been computed (for adiabaticity analysis)
#nth item in list corresponds to nth cross section (currently all are the same)
xmin = 117
xmax = 134
ymin = 64
ymax = 78

#neutrino momentum
p_abs = 1e7 # eV



In [ ]:
######################################
### PART 1: 3-FLAVOR, 3D SPIN-FLIP ###
######################################
location=[128,71,15] 

emu_filename = emu_data_loc + "i{:03d}".format(location[0])+"_j{:03d}".format(location[1])+"_k{:03d}".format(location[2])+"/allData.h5"

#compute relevant SpinParams object for the given point
SP_0 = sft.SpinParams(0,
                      emu_filename,
                      merger_data_filename,
                      location,
                      p_abs,
                      resonance_type = 'simplified',
                      gradient_filename = gradient_filename)


In [ ]:
### Angular plot of Eigenvector Resonance value (1-|L-R|) and zoomed in version of the same ###
resolution = 400
data = SP_0.angularEigenvectorPlot(resolution, 400,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.33,
                            zoom = None,
                            resonant_band_thetas=[2.341995193081677,2.3178148626917529,2.332116965261659],
                            shift = [0,0],
                            vmax = -11.0, vmin = -14.0,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            method = 'Nelder-Mead',
                            bounds =[(np.pi/2, np.pi)], 
                            linearPlot = False,
                            savefig = plot_output_file + "angular_resonance_init",
                            fs = 20)



In [ ]:
### Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###
# this identifies resonance bands in the channels given in flavor-resonances with the given colors. There are 9 resonance bands total, and we can show 3
# after showing 3 resonance bands we can plot their evolution in the next plot to show they are resonant.
resolution = 350
SP_0.linearEigenvectorPlot( 6*resolution,  
                            vmax = 2,
                            plot_bounds = [0.5, np.pi],
                            initvector = None, value = 'Omega',
                            zoom_on_vector = np.pi/2,
                            separations_from_peak = [0.06,0.06,0.03],
                            zoom =  np.pi/2, shift = 0, phi_optimal= np.pi,
                            method = 'Nelder-Mead',
                            legend_loc = (1.01,0.73),
                            bounds =[(np.pi/2, np.pi)],
                            extra_lines = None, extra_init_vectors = None,
                            flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_init',
                            fs = 20)

In [ ]:
### Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###
# this identifies resonance bands in the channels given in flavor-resonances with the given colors. There are 9 resonance bands total, and we can show 3
# after showing 3 resonance bands we can plot their evolution in the next plot to show they are resonant.
resolution = 350
SP_0.linearEigenvectorPlot( 6*resolution,  
                            vmax = 2,
                            #plot_bounds = [0.5, np.pi],
                            max_point = True,
                            initvector = None, value = 'Omega',
                            zoom_on_vector = 0.8072753906250002,
                            separations_from_peak = [0.06,0.06,0.03],
                            zoom =  2, shift = 0, phi_optimal= np.pi,
                            method = 'Nelder-Mead',
                            legend_loc = (1.01,0.73),
                            bounds =[(0.5, 1)],
                            extra_lines = None, extra_init_vectors = None,
                            flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_init',
                            fs = 20)

In [ ]:
### Single-Point Adiabaticity plot: shows adiabaticity and gradient vs azimuthal angle over resonance band for selected point ###
SP_0.azimuthalGradientsPlot(phi_resolution = 1000,  
                            vmax = 10,
                            savefig = plot_output_file + 'azimuthal_gradients_init')



### Adiabaticity width for single-point: prints and returns the angular width of the adiabatic region on the resonance band at the given point
# We'll quote this value when describing the plot
adiabatic_width = SP_0.findAdiabaticRegions(phi_resolution = 300,
                          min_dist_between_peaks = 1,
                          adiabaticity_threshold = 1,
                          max_peak_count = 2, 
                          method = 'Nelder-Mead')

print('total initial adiabatic width = ', adiabatic_width)
#print('total initial resonant and adiabatic solid angle = ' , solid_angle* adiabatic_width/(2*np.pi))



In [ ]:
### Show components of H to connect resonance bands to crossings ###
SP_0.H_component_plotter(0,
                         [0,1,3,4],
                         300,
                         xlims = [[0,np.pi]],
                         ylims = [[-6,6]],
                         legend_loc = (0.53,0.64),
                         savefig = plot_output_file + 'H_components_init',)

In [ ]:
### Compute resonant thetas on resonance bands for the next plot, and find the resonant angular width according to generalized resonance condition ###
find_res_regions = SP_0.findResonantRegions(theta_resolution = 1000, 
                                            phi_optimal = np.pi,
                                            min_dist_between_peaks = 20,
                                            limits = [0,np.pi],
                                            resonance_threshold = 1,
                                            max_peak_count = 3,
                                            method = 'Nelder-Mead',
                                            makeplot = False,
                                            printvalues = True)
# print total resonant width that will be used in description of the plot
resonant_width = find_res_regions['total_width']
resonant_thetas = find_res_regions['resonant_thetas']
solid_angle = 2*np.pi*resonant_width*np.sin(resonant_thetas[1])
print('total initial resonant width = ', resonant_width)
print('total initial resonant solid angle = ' , solid_angle) 


In [ ]:
### compute resonant Hamiltonians and find Timescales
# first compute the resonant thetas, vectors and Hamiltonians corresponding to these thetas
resonant_states = SP_0.resonant_states(resonant_thetas, P = True)
resonant_Hs = np.array([SP_0.H(float(theta),np.pi) for theta in resonant_thetas])

for H in resonant_Hs: #find oscillation timescale
    D = dg.Diagonalizer(H)
    timescale = D.timescale/2
    lengthscale = c*timescale/100/1000
    print('timescale = ' + str(timescale) + ' seconds')
    print('lengthscale = ' + str(lengthscale) + ' km')

In [ ]:
###############################
### PART 2: FFI SIMULATION  ###
###############################

In [ ]:
### time plot of H_LR increasing during instability in z direction ###
TP = tp.TimePlots([0,150], emu_filename,  merger_data_filename, location, p_abs)

TP.plot( quantity = 'H_LR_components', 
        thetas_and_phis = [(0.005,0),(np.pi/2,0)], 
        labels = (r'$\theta = 0$',r'$\theta = \pi/2$'),
        savefig = plot_output_file + 'hlr_time_evolution',
        vmax = [None, 6E-12])


In [ ]:
### Compute evolved SP object, print time of new plots
#try to load for all values of t until one doesnt return an error
SP_e = sft.SpinParams(250,
                      emu_filename,
                      merger_data_filename,
                      location, p_abs,
                      gradient_filename = gradient_filename_end)
print('time of new plots = ', SP_e.t_seconds*1E9)


In [ ]:
### Evolved angular eigenvector plot showing resonant bands after FFI
SP_e.angularEigenvectorPlot(200,20,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.4,
                            zoom = None,
                            shift = [0,0],
                            vmax = -11, vmin = -14.0,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            method = 'Nelder-Mead',
                            bounds =[(np.pi/2, np.pi)], 
                            linearPlot = False,
                            flavor_resonances=[],
                            savefig = plot_output_file + "angular_resonance_final.png",
                            fs = 20)

In [ ]:
resolution = 100
### Evolved Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###
#try to load for all values of t until one doesnt return an error

SP_e.linearEigenvectorPlot( 20*resolution,  
                            initvector = None, value = 'Omega',
                            #plot_bounds=[0,np.pi],
                            zoom_on_vector =1.504921875,
                            zoom = 0.002, shift = 0, phi_optimal= np.pi,
                            max_point = True,
                            method = 'Nelder-Mead',
                            vmax = 20000,
                            legend_loc = (1.01,0.73),
                            bounds =[(1.50, 1.51)],
                            #extra_lines = resonant_thetas, extra_init_vectors = None,
                            flavor_resonances = None,
                            # flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_final',
                            fs = 20)

In [ ]:
### Compute resonant thetas on resonance bands for the next plot, and find the resonant angular width according to generalized resonance condition ###
find_res_regions = SP_e.findResonantRegions(theta_resolution = 10000, 
                                            phi_optimal = np.pi,
                                            min_dist_between_peaks = 20,
                                            limits = [0, np.pi],
                                            resonance_threshold = 1,
                                            max_peak_count = 9,
                                            method = 'Nelder-Mead',
                                            makeplot = False,
                                            printvalues = True)
# print total resonant width that will be used in description of the plot
resonant_width = find_res_regions['total_width']
resonant_thetas = find_res_regions['resonant_thetas']
solid_angle = 2*np.pi*resonant_width*np.sin(resonant_thetas[1])
print('total initial resonant width = ', resonant_width)
print('total initial resonant solid angle = ' , solid_angle) 


In [ ]:
resolution = 100
### Evolved Linear plot of Eigenvector resonance value vs theta for phi = 3/4 pi (shows 1D phi-slice of zoomed in picture from previous plot)###
#try to load for all values of t until one doesnt return an error
location=[124,70,15] 
SP_e = sft.SpinParams(200,
                      emu_filename,
                      merger_data_filename,
                      location, p_abs,
                      gradient_filename = gradient_filename_end)
print('time of new plots = ', SP_e.t_seconds*1E9)
SP_e.linearEigenvectorPlot( 20*resolution,  
                            initvector = None, value = 'Omega',
                            plot_bounds=[0,np.pi],
                            zoom_on_vector = 2.4,
                            zoom = 0.32, shift = 0, phi_optimal= np.pi,
                            method = 'Nelder-Mead',
                            vmax = 1,
                            legend_loc = (1.01,0.73),
                            bounds =[(np.pi/2, np.pi)],
                             extra_init_vectors = None,
                            flavor_resonances = None,
                            # flavor_resonances = [(0,0,'deepskyblue'), (2,2,'limegreen'), (0,2,'magenta')],
                            savefig = plot_output_file + 'linear_eigenvector_plot_final',
                            fs = 20)

In [ ]:
###########################################
###### PART 3: FULL MERGER ANALYSIS #######
###########################################

In [ ]:
### Angular Plot of H_LR, before and after simulation ###
sft.multi_HLR_Plotter( 
                    0, 250,
                    emu_filename,
                    merger_data_filename,
                    location,
                    p_abs,
                    40,
                    30, 
                    resonance_type = 'simplified',
                    savefig= plot_output_file + 'angular_hlr_init',
                    use_gm=[1,2],
                    )


In [ ]:
#compute data for the four angular plots in the resonance multiplot
resolution = 700
emu_data_loc = "/mnt/scratch/shared/3-Henry_NSM_box/"
locations = [[105,130,98],
             [122,130,99],
             [83,76,98],
             ]
timesteps = [0,250]

emu_filenames = []
for i in range(len(locations)):
    emu_filenames.append(emu_data_loc + 
                         "i{:03d}".format(locations[i][0])   \
                         +"_j{:03d}".format(locations[i][1]) \
                         +"_k{:03d}".format(locations[i][2])
                         +"/allData.h5")
SP_list = []
for i in range((len(locations))):
    for j in range(2):
        SP_list.append(sft.SpinParams(timesteps[j],
                                  emu_filenames[i],
                                  merger_data_filename,
                                  locations[i],
                                  p_abs,
                                  resonance_type = 'simplified',
                                  gradient_filename = None))

#data_list = []
for SP in SP_list[3:]:
    data_list2.append(SP.angularEigenvectorPlot(resolution, resolution,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.33,
                            returndata = True,
                            vmax = -9.256, vmin = -14.035,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            fs = 20))



In [ ]:
#store data list in h5 file
f = h5py.File(plot_output_file + 'angular_resonance_init_data.h5', 'w')
for i in range(len(data_list)):
    f.create_dataset('data'+str(i), data = data_list[i][1])

In [ ]:
np.pi

In [ ]:
data_list2 = [[2.33,[],None,None,None]]*3

In [ ]:
data_list2

In [ ]:
#extract data from h5 file
f = h5py.File(plot_output_file + 'angular_resonance_init_data.h5', 'r')
for i in range(3):
    data_list2[i][1] = (f['data'+str(i)][:])

In [ ]:
data_list

In [ ]:
### Resonance and FFI Regions Plot: plots resonance and FFI regions on z-cross sections ###
points = [location] + locations
#points[0][2] = 1
#points[3][0]+=3
#points[4][1]-=2
for n in range(1,len(points)):
    points[n][2] = 2
print(points)
mg.Merger_Grid([1,15, 98],
               merger_data_loc=merger_data_filename,
               p_abs=p_abs
                ).contour_plot(
                            xmin=20, xmax = 180, ymin=20, ymax=180,
                            points = points,
                            rect_xmin = [None, location[0]-9,None], rect_xmax = [None, location[0]+8,None],
                            rect_ymin = [None, location[1]-5,None], rect_ymax = [None, location[1]+9,None],
                            savefig = plot_output_file + 'merger_grid_init')


In [ ]:
#compute data for the four angular plots in the resonance multiplot
resolution = 100
emu_data_loc = "/mnt/scratch/shared/3-Henry_NSM_box/"
locations = [
             [122,130,99],
             ]
timesteps = [0,250]

emu_filenames = []
for i in range(len(locations)):
    emu_filenames.append(emu_data_loc + 
                         "i{:03d}".format(locations[i][0])   \
                         +"_j{:03d}".format(locations[i][1]) \
                         +"_k{:03d}".format(locations[i][2])
                         +"/allData.h5")
SP_list = []
for i in range((len(locations))):
    for j in range(2):
        SP_list.append(sft.SpinParams(timesteps[j],
                                  emu_filenames[i],
                                  merger_data_filename,
                                  locations[i],
                                  p_abs,
                                  resonance_type = 'simplified',
                                  gradient_filename = None))

data_list = []
for SP in SP_list:
    data_list.append(SP.angularEigenvectorPlot(resolution, 20,
                            value = 'Omega',
                            phi_optimal = np.pi, theta_optimal=2.33,
                            returndata = True,
                            vmax = -9.256, vmin = -14.035,
                            zoom_resolution = resolution,
                            initvector = 'default', 
                            fs = 20))



In [ ]:
###  find the resonant angular width at each of 4 example points according to generalized resonance condition ###


find_res_regions = SP.findResonantRegions(theta_resolution = 300, 
                                        phi_optimal = np.pi,
                                        min_dist_between_peaks = 20,
                                        limits = [0,np.pi],
                                        resonance_threshold = 1,
                                        max_peak_count = 6,
                                        method = 'Nelder-Mead',
                                        makeplot = False,
                                        printvalues = True)
# print total resonant width that will be used in description of the plot
resonant_width = find_res_regions['total_width']
resonant_thetas = find_res_regions['resonant_thetas']
#solid_angle = 2*np.pi*resonant_width*np.sin(resonant_thetas[1])
print('total initial resonant width = ', resonant_width)
#print('total initial resonant solid angle = ' , solid_angle) 


In [ ]:
### resonance band multiplot showing general distribution of bands for 2 selected points
sft.angular_eigenvector_multiplot(data_list, vmax = -9.256, vmin = -14.035,
                                  savefig = plot_output_file + 'angular_resonance_multiplot')

In [ ]:
### plot histogram of HLR values

#load data
#emu_data_full = "/mnt/scratch/shared/3-Henry_NSM_box_initial/"
#HLR_list = sft.HLR_list(emu_data_full,merger_data_filename, p_abs)
# save HLR list as h5 file
#h5file = h5py.File('/mnt/scratch/shared/HLR_list.h5', 'w')
#h5file.create_dataset('HLR_list', data=HLR_list)
#h5file.close()

#plot
HLR_list = h5py.File('/mnt/scratch/shared/HLR_list.h5',"r")['HLR_list'][()]
f, ax = plt.subplots(figsize=(8,6))
N, bins, patches = ax.hist(np.array(HLR_list)*1E10, bins=100,
         log = True, density=False, 
         cumulative=False,
        )
for patch in patches:
        patch.set_facecolor('white')

  
#color the patch corresponding to example point
HLR_example = np.abs(SP_0.H_LR(np.pi/2,0)[0,0])
index_example = np.where(bins>HLR_example*1E10)[0][0]
patches[index_example].set_facecolor('lime')

#color the patches with length scales greater than 10 km 
maxlengthscale = 20E3 #meters
energy_limit = hbar*np.pi*c/100/maxlengthscale/2
indices_limit = np.where(bins<energy_limit*1E10)[0]

toobig_sum = 0
for i in indices_limit:
    toobig_sum += N[i]
patches[indices_limit[-1]].set_facecolor('red')
print (f'fraction of cells with |H_LR| > {energy_limit*1E10} eV: {toobig_sum/len(HLR_list)}')
N, bins, bg = ax.hist(np.array(HLR_list)*1E10, bins=100,
         log = True, density=False, 
         cumulative=False,
         histtype='step',
         color = 'blue',
        )


ax.set_xlabel(r'$|H_{LR}^{ee}|$ (eV $\times 10^{-10}$)')
ax.set_ylabel('Cell Count')


f.savefig(plot_output_file + 'HLR_histogram.pdf', dpi=300, bbox_inches='tight')
h5py.File('/mnt/scratch/shared/HLR_list.h5',"r").close()
print(f'required energy for cell width: {hbar*np.pi*c/100/0.67e3/2}')


In [ ]:
#self.H_vv_min = 2**(1./2.)*G*(self.J[0]+self.mag_F)
#        self.H_vv_max = 2**(1./2.)*G*(self.J[0]-self.mag_F)

#min occurs when flux is along basis direction
     #   self.resonance_val_min = 2**(-1/2)*G*self.n_b*(3.*self.Ye-np.ones_like(self.Ye))+self.H_vv_max
     #   self.resonance_val_max = 2**(-1/2)*G*self.n_b*(3.*self.Ye-np.ones_like(self.Ye))+self.H_vv_min


In [ ]:
J

In [ ]:
### Solid Angle Plot: shows resonant and adiabatic solid angle at each point in region of interest
emu_data_bestpoints = "/mnt/scratch/shared/3-Henry_NSM_box_bestpoints/"

data = sft.solidAnglePlot([xmin,xmax], [ymin,ymax], location[2],
                    0, 250,
                    emu_data_bestpoints,
                    merger_data_filename, 
                    [gradient_filename, gradient_filename_end],
                    p_abs,
                    -16, -14,
                    marker =[129,72,15][:2],
                    separate_ranges = False,
                    resonance_type = None,
                    adiabaticity = True,
                    data = data,
                    plot_conditions=True,
                    savefig = False)



In [ ]:
sft.HLR_magnitude_plotter([[xmin,xmax,],[ymin,ymax]], #[[x1,x2],[y1,y2]]
                          15, #z location
                          0, #timestep
                          [0,0], #quantity to plot. can be 'GM_magnitude' or a component of H_LR [a,b]
                          np.pi/2, 0, #direction
                          emu_data_loc,
                          merger_data_filename,
                          p_abs,
                          -14, -9,
                          mark_point = [126,70],
                          gradient_filename = None,
                          savefig = False)